In [1]:
#Setting up the code and installing required packages 
%load_ext autotime
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import LineString, Polygon
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
from shapelysmooth import taubin_smooth
pd.set_option("display.max_columns", None)

time: 1.74 s (started: 2024-08-07 16:40:57 +12:00)


In [2]:
#Navigate to intersects shapefile and separates out the year, finds how many years from 1800 and how many years to 2100
gdf = gpd.read_file(f"Data/Merged Intersects_UniqueID/NorthIsland_Intersects.shp")
gdf["Date"] = pd.to_datetime(gdf.ShorelineI, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf["TransectID"] = gdf.Unique_ID.astype(np.int64)
gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture,TransectID
0,01/03/2011,0.0,-37.16,1.730591e+06,5.908710e+06,8.59,1.006327e+11,2011-03-01,POINT Z (1730590.982 5908710.362 0.000),2011,211.156742,88.837782,100632729920
1,08/19/1960,0.0,-51.70,1.730600e+06,5.908699e+06,3.76,1.006327e+11,1960-08-19,POINT Z (1730600.339 5908699.234 0.000),1960,160.626968,139.367556,100632729920
2,01/04/2017,0.0,-32.09,1.730588e+06,5.908714e+06,2.94,1.006327e+11,2017-04-01,POINT Z (1730587.720 5908714.241 0.000),2017,217.242984,82.751540,100632729920
3,01/02/2004,0.0,-48.27,1.730598e+06,5.908702e+06,10.05,1.006327e+11,2004-02-01,POINT Z (1730598.132 5908701.858 0.000),2004,204.079398,95.915127,100632729920
4,01/04/2022,0.0,-23.18,1.730582e+06,5.908721e+06,2.07,1.006327e+11,2022-04-01,POINT Z (1730581.983 5908721.063 0.000),2022,222.242300,77.752225,100632729920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914496,02/21/2021,0.0,-104.61,1.800540e+06,5.395670e+06,1.02,1.004660e+11,2021-02-21,POINT Z (1800539.730 5395670.271 0.000),2021,221.136208,78.858316,100466049969
914497,09/24/1995,0.0,-69.89,1.800538e+06,5.395636e+06,5.64,1.004660e+11,1995-09-24,POINT Z (1800538.365 5395635.582 0.000),1995,195.723477,104.271047,100466049969
914498,04/12/1973,0.0,-46.92,1.800527e+06,5.395613e+06,4.73,1.004661e+11,1973-12-04,POINT Z (1800527.469 5395613.018 0.000),1973,173.919233,126.075291,100466050980
914499,02/21/2021,0.0,-104.80,1.800530e+06,5.395671e+06,1.02,1.004661e+11,2021-02-21,POINT Z (1800529.745 5395670.850 0.000),2021,221.136208,78.858316,100466050980


time: 1min 34s (started: 2024-08-07 16:40:59 +12:00)


In [3]:
# Filter to just Karekare for testing
gdf = gdf.cx[1727077.5508:1735465.0761,5897869.0262:5910640.2418]
gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture,TransectID
0,01/03/2011,0.0,-37.16,1.730591e+06,5.908710e+06,8.59,1.006327e+11,2011-03-01,POINT Z (1730590.982 5908710.362 0.000),2011,211.156742,88.837782,100632729920
1,08/19/1960,0.0,-51.70,1.730600e+06,5.908699e+06,3.76,1.006327e+11,1960-08-19,POINT Z (1730600.339 5908699.234 0.000),1960,160.626968,139.367556,100632729920
2,01/04/2017,0.0,-32.09,1.730588e+06,5.908714e+06,2.94,1.006327e+11,2017-04-01,POINT Z (1730587.720 5908714.241 0.000),2017,217.242984,82.751540,100632729920
3,01/02/2004,0.0,-48.27,1.730598e+06,5.908702e+06,10.05,1.006327e+11,2004-02-01,POINT Z (1730598.132 5908701.858 0.000),2004,204.079398,95.915127,100632729920
4,01/04/2022,0.0,-23.18,1.730582e+06,5.908721e+06,2.07,1.006327e+11,2022-04-01,POINT Z (1730581.983 5908721.063 0.000),2022,222.242300,77.752225,100632729920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85101,04/05/2016,2.0,-288.76,1.731087e+06,5.904261e+06,8.84,1.006321e+11,2016-05-04,POINT Z (1731087.039 5904261.021 0.000),2016,216.334018,83.660507,100632146004
85102,04/07/2010,2.0,-284.87,1.731083e+06,5.904261e+06,8.84,1.006321e+11,2010-07-04,POINT Z (1731083.142 5904260.921 0.000),2010,210.499658,89.494867,100632146004
85103,04/30/2024,2.0,-347.31,1.731146e+06,5.904263e+06,8.83,1.006321e+11,2024-04-30,POINT Z (1731145.565 5904262.521 0.000),2024,224.323066,75.671458,100632146004
85104,05/18/2020,2.0,-297.29,1.731096e+06,5.904261e+06,8.84,1.006321e+11,2020-05-18,POINT Z (1731095.558 5904261.239 0.000),2020,220.372348,79.622177,100632146004


time: 173 ms (started: 2024-08-07 16:42:34 +12:00)


In [4]:
def get_transects(intersects):
  p1 = intersects.geometry[intersects.Distance.idxmin()].coords[0]
  p2 = intersects.geometry[intersects.Distance.idxmax()].coords[0]
  azimuth = math.degrees(math.atan2(p1[0]-p2[0], p1[1]-p2[1]))
  if azimuth < 0:
      azimuth += 360
  return pd.Series({"Azimuth": azimuth, "geometry": LineString([p1, p2])})

lines = gdf.groupby("TransectID")[["geometry", "Distance"]].apply(get_transects)
lines.crs = gdf.crs
lines

,Azimuth,geometry
TransectID,,
100631371502,336.037509,"LINESTRING Z (1733980.469 5899100.261 0.000, 1..."
100631372531,336.037511,"LINESTRING Z (1733970.210 5899098.722 0.000, 1..."
100631373548,336.037511,"LINESTRING Z (1733960.009 5899097.052 0.000, 1..."
100631374557,336.037509,"LINESTRING Z (1733949.874 5899095.233 0.000, 1..."
100631375562,336.037512,"LINESTRING Z (1733939.704 5899093.493 0.000, 1..."
...,...,...
100632934491,64.787380,"LINESTRING Z (1730302.467 5910628.607 0.000, 1..."
100632935491,64.787380,"LINESTRING Z (1730300.528 5910638.747 0.000, 1..."
100632936491,64.787381,"LINESTRING Z (1730277.720 5910639.062 0.000, 1..."


time: 886 ms (started: 2024-08-07 16:42:34 +12:00)


In [5]:
lines["dist_to_neighbour"] = lines.distance(lines.shift(-1))
breakpoints = lines.dist_to_neighbour[lines.dist_to_neighbour > 15]
lines["group"] = pd.Series(range(len(breakpoints)), index=breakpoints.index)
lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)
transect_metadata = lines[["Azimuth", "group"]].to_dict(orient="index")

time: 8.37 ms (started: 2024-08-07 16:42:35 +12:00)


In [6]:
transect_metadata

{100631371502: {'Azimuth': 336.03750912910954, 'group': 0},
 100631372531: {'Azimuth': 336.037511296291, 'group': 0},
 100631373548: {'Azimuth': 336.03751129632604, 'group': 0},
 100631374557: {'Azimuth': 336.0375088582414, 'group': 0},
 100631375562: {'Azimuth': 336.0375115672188, 'group': 0},
 100631376565: {'Azimuth': 336.03751129634435, 'group': 0},
 100631377566: {'Azimuth': 336.0375112963323, 'group': 0},
 100631378566: {'Azimuth': 336.03751156721285, 'group': 0},
 100631379566: {'Azimuth': 336.03750885824695, 'group': 0},
 100631380567: {'Azimuth': 336.0375112962853, 'group': 0},
 100631381571: {'Azimuth': 336.0375115672518, 'group': 0},
 100631382578: {'Azimuth': 336.03750912921737, 'group': 0},
 100631383591: {'Azimuth': 336.0375112963934, 'group': 0},
 100631384608: {'Azimuth': 336.0375112963167, 'group': 0},
 100631385632: {'Azimuth': 336.03751156717647, 'group': 0},
 100631386663: {'Azimuth': 336.0375112962182, 'group': 0},
 100631387705: {'Azimuth': 336.0375088581859, 'gro

time: 40.2 ms (started: 2024-08-07 16:42:35 +12:00)


In [7]:
#Linear regression is run here. See util.py for the breakdown on linear_models
linear_models = fit(gdf, transect_metadata)
linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,100631371502,0.322174,-360.005759,0,0.017436,53.731284,3684.280899,60.698278
1,100631372531,0.347961,-367.567862,0,0.021097,52.636069,3538.726101,59.487193
2,100631373548,0.358693,-372.305454,0,0.022749,52.179073,3481.469452,59.003978
3,100631374557,0.376138,-377.804140,0,0.025176,51.878670,3450.686103,58.742541
4,100631375562,0.382848,-380.962159,0,0.026495,51.279141,3392.274845,58.243239
...,...,...,...,...,...,...,...,...
1007,100632934491,0.789328,-206.874403,10,0.806864,6.843583,72.634645,8.522596
1008,100632935491,0.684289,-185.947375,10,0.733430,7.642753,98.091294,9.904105
1009,100632936491,0.668656,-181.851348,10,0.742733,6.046609,58.287373,7.634617
1010,100632937373,1.495830,-357.084255,10,0.605602,5.170501,41.373596,6.432231


time: 1.47 s (started: 2024-08-07 16:42:35 +12:00)


In [8]:
#Only run if rolling average is needed. Otherwise SKIP THIS
#linear_models = fit(gdf, transect_metadata)
#rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
#linear_models.slope = rolled_slopes.slope
#linear_models.dropna(inplace=True)
#linear_models

time: 361 µs (started: 2024-08-07 16:42:36 +12:00)


In [9]:
#Coordinates of the projected shoreline are plotted here

#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

time: 1.3 ms (started: 2024-08-07 16:42:36 +12:00)


In [10]:
#Projection file is created here with the stats and coordinate points in table format
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.006314e+11,0.322174,-360.005759,0.0,0.017436,53.731284,3684.280899,60.698278,3.0,2100,POINT (1734202.2877831638 5898601.168385305),POINT (1734038.3444386926 5898970.04087747),-263.353642,-96.336358
1,1.006314e+11,0.347961,-367.567862,0.0,0.021097,52.636069,3538.726101,59.487193,3.0,2100,POINT (1734193.076361982 5898597.271925394),POINT (1734029.2768926786 5898965.820736022),-263.179425,-96.690575
2,1.006314e+11,0.358693,-372.305454,0.0,0.022749,52.179073,3481.469452,59.003978,3.0,2100,POINT (1734181.6203083994 5898598.425901516),POINT (1734019.5232864474 5898963.144205555),-264.697634,-100.882366
3,1.006314e+11,0.376138,-377.804140,0.0,0.025176,51.878670,3450.686103,58.742541,3.0,2100,POINT (1734170.8145996262 5898598.116615453),POINT (1734010.279180196 5898959.321272358),-264.962596,-104.727404
4,1.006314e+11,0.382848,-380.962159,0.0,0.026495,51.279141,3392.274845,58.243239,3.0,2100,POINT (1734161.456681661 5898594.549771212),POINT (1734000.675510183 5898956.307416256),-266.107732,-104.122268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,1.006329e+11,0.789328,-206.874403,10.0,0.806864,6.843583,72.634645,8.522596,3.0,2100,POINT (1729784.4293691465 5910384.697912364),POINT (1730192.2342017298 5910576.706028557),29.924065,-49.254065
1008,1.006329e+11,0.684289,-185.947375,10.0,0.733430,7.642753,98.091294,9.904105,3.0,2100,POINT (1729785.5240314973 5910396.266297602),POINT (1730197.5582886615 5910590.265767954),19.339289,-44.579289
1009,1.006329e+11,0.668656,-181.851348,10.0,0.742733,6.046609,58.287373,7.634617,3.0,2100,POINT (1729778.002671802 5910403.777982806),POINT (1730193.8313230874 5910599.563970376),18.745357,-40.385357
1010,1.006329e+11,1.495830,-357.084255,10.0,0.605602,5.170501,41.373596,6.432231,3.0,2100,POINT (1729764.1645612943 5910429.2493781885),POINT (1730122.478100242 5910581.005699454),91.664605,-110.874605


time: 1.97 s (started: 2024-08-07 16:42:36 +12:00)


In [11]:
#Spatial reference added to the results
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.006314e+11,0.322174,-360.005759,0.0,0.017436,53.731284,3684.280899,60.698278,3.0,2100,POINT (1734202.2877831638 5898601.168385305),POINT (1734038.344 5898970.041),-263.353642,-96.336358
1,1.006314e+11,0.347961,-367.567862,0.0,0.021097,52.636069,3538.726101,59.487193,3.0,2100,POINT (1734193.076361982 5898597.271925394),POINT (1734029.277 5898965.821),-263.179425,-96.690575
2,1.006314e+11,0.358693,-372.305454,0.0,0.022749,52.179073,3481.469452,59.003978,3.0,2100,POINT (1734181.6203083994 5898598.425901516),POINT (1734019.523 5898963.144),-264.697634,-100.882366
3,1.006314e+11,0.376138,-377.804140,0.0,0.025176,51.878670,3450.686103,58.742541,3.0,2100,POINT (1734170.8145996262 5898598.116615453),POINT (1734010.279 5898959.321),-264.962596,-104.727404
4,1.006314e+11,0.382848,-380.962159,0.0,0.026495,51.279141,3392.274845,58.243239,3.0,2100,POINT (1734161.456681661 5898594.549771212),POINT (1734000.676 5898956.307),-266.107732,-104.122268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,1.006329e+11,0.789328,-206.874403,10.0,0.806864,6.843583,72.634645,8.522596,3.0,2100,POINT (1729784.4293691465 5910384.697912364),POINT (1730192.234 5910576.706),29.924065,-49.254065
1008,1.006329e+11,0.684289,-185.947375,10.0,0.733430,7.642753,98.091294,9.904105,3.0,2100,POINT (1729785.5240314973 5910396.266297602),POINT (1730197.558 5910590.266),19.339289,-44.579289
1009,1.006329e+11,0.668656,-181.851348,10.0,0.742733,6.046609,58.287373,7.634617,3.0,2100,POINT (1729778.002671802 5910403.777982806),POINT (1730193.831 5910599.564),18.745357,-40.385357
1010,1.006329e+11,1.495830,-357.084255,10.0,0.605602,5.170501,41.373596,6.432231,3.0,2100,POINT (1729764.1645612943 5910429.2493781885),POINT (1730122.478 5910581.006),91.664605,-110.874605


time: 18.5 ms (started: 2024-08-07 16:42:38 +12:00)


In [12]:
#Line and polygon shapefiles are created here 
def prediction_results_to_line_polygon(results: gpd.GeoDataFrame):
    lines = []
    polygons = []
    for group_name, group_data in results.groupby(["BaselineID", "group"]):
        if len(group_data) > 1:
            # Convert the points to LineString
            line = LineString(list(group_data.geometry))
            lines.append(line)
            # Convert the points to a closed Polygon
            polygon = Polygon(list(group_data.geometry) + list(group_data.ocean_point)[::-1])
            polygons.append(polygon)
    lines = gpd.GeoSeries(lines, crs=2193)
    polygons = gpd.GeoSeries(polygons, crs=2193)
    return lines, polygons
lines, poly = prediction_results_to_line_polygon(results)

time: 69.4 ms (started: 2024-08-07 16:42:38 +12:00)


In [17]:
m = lines.explore()
lines.apply(lambda line: taubin_smooth(line, steps=500)).explore(m=m, color="red")

time: 41.5 ms (started: 2024-08-07 16:46:07 +12:00)


In [15]:
#Saving line and polygon projection file to Z drive. Change file location accordingly  
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Z:\Lalita\RNC Cont\......\BigBay_projection_output_lines.shp")
poly.to_file("Z:\Lalita\RNC Cont\.......\BigBay_projection_output_polygon.shp")

DriverIOError: Failed to create file Z:\Lalita\RNC Cont\....../BigBay_projection_output_lines.shp: No such file or directory

time: 859 ms (started: 2024-08-07 16:42:39 +12:00)


In [ ]:
#Saving line and polygon projection file to folder in VS Code. Change file location accordingly
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Projections\NorthIsland_projection_output_lines.shp")
poly.to_file("Projections\NorthIsland_projection_output_polygon.shp")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 11-12: malformed \N character escape (3394349576.py, line 3)

In [ ]:
#Quick visualisation of projected polygon and historic shorelines 
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)